# Import des données de comptage

In [110]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import Connexion_Transfert as ct
import Import_trafics as it
import Outils
from sqlalchemy.schema import MetaData

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## CD 23
- Année 2018 : 
> à partir du fichier __Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD23\2018_CD23_trafics.xls__<br>***attention : pour le point de comptage D941 6+152 à Aubusson, le pR est 32 et non 6. il faut donc corriger à la main le fihcier excel***
<br> Pour le moment tous les points sont déjà dans  la base, dc pas de traitement de type insert prévus.

In [11]:
# ouvrir le classeur
df_excel=pd.read_excel(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD23\2018_CD23_trafics.xls',skiprows=11)
# renomer les champs
df_excel_rennome=df_excel.rename(columns={'1er trimestre  du 01 janvier au 31 mars':'trim1_TV', 'Unnamed: 9':'trim1_pcpl',
                         '2ème trimestre du 01 avril au 30 juin':'trim2_TV', 'Unnamed: 11':'trim2_pcpl',
                         '3ème trimestre du 01 juillet au 30 septembre':'trim3_TV', 'Unnamed: 13':'trim3_pcpl',
                         '4ème trimestre du 01 octobre au 31 décembre':'trim4_TV', 'Unnamed: 15':'trim4_pcpl',
                         'Unnamed: 17':'pc_pl', 'TMJA 2018':'tmja'})
#supprimer la 1ere ligne
df_excel_filtre=df_excel_rennome.loc[1:,:].copy()
#mise en forme attribut
df_excel_filtre['Route']=df_excel_filtre.apply(lambda x : str(x['Route']).upper(), axis=1)
annee_cpt='2018'
#attribut id_comptag
for i in ['DEP','PR','ABS'] : 
    df_excel_filtre[i]=df_excel_filtre.apply(lambda x : str(int(x[i])),axis=1)
df_excel_filtre['id_comptag']=df_excel_filtre.apply(lambda x : '-'.join([x['DEP'],'D'+str(x['Route']),
                                                                         x['PR']+'+'+x['ABS']]),axis=1)
#donnees_mensuelles
list_id_comptag=[val for val in df_excel_filtre.id_comptag.tolist() for _ in (0, 1)]
donnees_type=['tmja','pc_pl']*len(df_excel_filtre.id_comptag.tolist())
annee_df=['2018']*2*len(df_excel_filtre.id_comptag.tolist())
janv, fev, mars,avril,mai,juin,juil,aout,sept,octo,nov,dec=[],[],[],[],[],[],[],[],[],[],[],[]
for i in range(len(df_excel_filtre.id_comptag.tolist())) :
    for j in (janv, fev, mars) :
        j.extend([df_excel_filtre.trim1_TV.tolist()[i],df_excel_filtre.trim1_pcpl.tolist()[i]])
    for k in (avril,mai,juin) :
        k.extend([df_excel_filtre.trim2_TV.tolist()[i],df_excel_filtre.trim2_pcpl.tolist()[i]])
    for l in (juil,aout,sept) :
        l.extend([df_excel_filtre.trim3_TV.tolist()[i],df_excel_filtre.trim3_pcpl.tolist()[i]])
    for m in (octo,nov,dec) :
        m.extend([df_excel_filtre.trim4_TV.tolist()[i],df_excel_filtre.trim4_pcpl.tolist()[i]])
donnees_mens=pd.DataFrame({'id_comptag':list_id_comptag,'donnees_type':donnees_type,'annee':annee_df,'janv':janv,'fevr':fev,'mars':mars,'avri':avril,
              'mai':mai,'juin':juin,'juil':juil,'aout':aout,'sept':sept,'octo':octo,'nove':nov,'dece':dec})

In [12]:
#Mise à jour bdd
with ct.ConnexionBdd('gti_otv') as c :
    c.curs.execute("select distinct id_comptag from comptage.na_2010_2018_p where dep='23' order by id_comptag")
    listerecord=[record[0] for record in c.curs]
    for id_comptag,tmja, pc_pl  in zip(df_excel_filtre.id_comptag.tolist(), df_excel_filtre.tmja.tolist(),df_excel_filtre.pc_pl.tolist()) : 
        if id_comptag in listerecord :
            c.curs.execute("update comptage.na_2010_2018_p set tmja_2018=%s, pc_pl_2018=%s, ann_cpt=%s where id_comptag=%s",(tmja, pc_pl,annee_cpt,id_comptag))
        else : 
            print (f'{id_comptag} nouveau, à traiter')
    print('fini')
    c.connexionPsy.commit()
    donnees_mens.to_sql('na_2010_2018_mensuel', c.sqlAlchemyConn,schema='comptage',if_exists='append',index=False)

23-D941-77+25 ok, MaJ
23-D941-60+270 ok, MaJ
23-D941-57+639 ok, MaJ
23-D941-24+257 ok, MaJ
23-D914-0+600 ok, MaJ
23-D940-39+944 ok, MaJ
23-D997-24+86 ok, MaJ
23-D915-5+595 ok, MaJ
23-D951-8+150 ok, MaJ
23-D990-59+690 ok, MaJ
23-D942-1+585 ok, MaJ
23-D942-23+698 ok, MaJ
23-D1-14+725 ok, MaJ
23-D1-22+760 ok, MaJ
23-D912-15+739 ok, MaJ
23-D5-35+590 ok, MaJ
23-D913-9+780 ok, MaJ
23-D951-28+425 ok, MaJ
23-D6-20+56 ok, MaJ
23-D6-0+826 ok, MaJ
23-D940-50+116 ok, MaJ
23-D940-66+431 ok, MaJ
23-D990-16+988 ok, MaJ
23-D942-31+637 ok, MaJ
23-D990-27+817 ok, MaJ
23-D990-45+295 ok, MaJ
23-D11-6+857 ok, MaJ
23-D11-22+776 ok, MaJ
23-D917-7+372 ok, MaJ
23-D917-12+491 ok, MaJ
23-D993-4+638 ok, MaJ
23-D915-18+907 ok, MaJ
23-D997-35+417 ok, MaJ
23-D996-21+935 ok, MaJ
23-D4-82+80 ok, MaJ
23-D990-65+576 ok, MaJ
23-D941A-4+172 ok, MaJ
23-D982-1+210 ok, MaJ
23-D997-13+862 ok, MaJ
23-D982-17+780 ok, MaJ
23-D982-36+433 ok, MaJ
23-D8-33+506 ok, MaJ
23-D912-42+22 ok, MaJ
23-D4-19+290 ok, MaJ
23-D4-35+917 ok, MaJ


## CD 40
> Année 2018 : contient des données mensuelless

In [ ]:
it.cd40()

## CD17
> Année 2015 : traitée dans le fichier Import_trafics.py

In [104]:
#ouverture du fichier
cpt17=it.comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_produites\donnnees_travail\Comptage\17\cpt_pctuel_but_2016.txt',
                       'brochure',2016)

In [129]:
cpt17.df_attr_insert

,route,pr,abs,tmja_2016,pc_pl_2016,id_comptag,dep,reseau,gestionnai,concession,type_poste,obs_2016
0,D0001,7,80,4583,0.72,17-D1-7+80,17,RD,CD17,N,ponctuel,"nouveau point,06/09/2016-14/09/2016,v85_tv 74.09"
1,D0001,7,270,5510,1.14,17-D1-7+270,17,RD,CD17,N,ponctuel,"nouveau point,06/09/2016-14/09/2016,v85_tv 67.8"
2,D0002,14,553,2032,3.67,17-D2-14+553,17,RD,CD17,N,ponctuel,"nouveau point,12/01/2016-18/01/2016,v85_tv 102.79"
3,D0002,15,190,2216,2.82,17-D2-15+190,17,RD,CD17,N,ponctuel,"nouveau point,12/01/2016-18/01/2016,v85_tv 106.96"
4,D0006,7,260,3930,1.00,17-D6-7+260,17,RD,CD17,N,ponctuel,"nouveau point,30/07/2016-07/08/2016,v85_tv 66.39"
5,D0009,14,110,11171,7.64,17-D9-14+110,17,RD,CD17,N,ponctuel,"nouveau point,26/10/2016-06/11/2016,v85_tv 103.96"
6,D0014,1,200,10848,2.30,17-D14-1+200,17,RD,CD17,N,ponctuel,"nouveau point,31/05/2016-06/06/2016,v85_tv 65.85"
7,D0017,0,115,3701,4.76,17-D17-0+115,17,RD,CD17,N,ponctuel,"nouveau point,07/10/2016-13/10/2016,v85_tv 26.44"
8,D0017,0,459,1568,0.12,17-D17-0+459,17,RD,CD17,N,ponctuel,"nouveau point,07/10/2016-13/10/2016,v85_tv 44.02"
9,D0017,0,680,3089,0.81,17-D17-0+680,17,RD,CD17,N,ponctuel,"nouveau point,07/10/2016-12/10/2016,v85_tv 62.64"


In [ ]:
with ct.ConnexionBdd('local_otv') as c:
    cpt17.df_attr_insert.to_sql('na_2010_2017_p',c.sqlAlchemyConn,schema='comptage',if_exists='append', index=False )

In [128]:
tbl

'comptage.na_2010_2017_p'